In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt


In [ ]:
# Load the CSV file with header in row 3
df = pd.read_csv('your_file.csv', header=2)

# Display the first few rows to understand the structure
print(df.head())

# Convert the 'Interval' column to datetime
df['Interval'] = pd.to_datetime(df['Interval'])

# Sort the data by 'Interval' just in case it is not sorted
df = df.sort_values('Interval')

# Drop any duplicates or missing values if necessary
df = df.drop_duplicates()
df = df.dropna()

# Filter out unnecessary columns if you want to focus only on the 'Average' usage
# You can adjust this to focus on CPU or Memory specifically by filtering on `Name` or `Rule`
df_filtered = df[['Interval', 'Name', 'Average']]

# Display cleaned data
print(df_filtered.head())


In [ ]:
# Example: Filter for CPU data
cpu_data = df_filtered[df_filtered['Name'].str.contains('CPU', case=False)]

# Example: Filter for Memory data
memory_data = df_filtered[df_filtered['Name'].str.contains('Memory', case=False)]


In [ ]:
# Set 'Interval' as the index for time series analysis
cpu_data.set_index('Interval', inplace=True)
memory_data.set_index('Interval', inplace=True)

# Split the data into training and testing sets (for example, 80% training, 20% testing)
train_cpu, test_cpu = train_test_split(cpu_data['Average'], test_size=0.2, shuffle=False)
train_memory, test_memory = train_test_split(memory_data['Average'], test_size=0.2, shuffle=False)


In [ ]:
# For CPU usage forecasting
cpu_model = ExponentialSmoothing(train_cpu, trend='add', seasonal='add', seasonal_periods=12)
cpu_model_fit = cpu_model.fit()
cpu_forecast = cpu_model_fit.forecast(steps=1)

print(f"CPU Usage Forecast for the next day: {cpu_forecast}")

# For Memory usage forecasting
memory_model = ExponentialSmoothing(train_memory, trend='add', seasonal='add', seasonal_periods=12)
memory_model_fit = memory_model.fit()
memory_forecast = memory_model_fit.forecast(steps=1)

print(f"Memory Usage Forecast for the next day: {memory_forecast}")


In [ ]:
# Evaluate CPU usage model
cpu_predictions = cpu_model_fit.forecast(steps=len(test_cpu))
cpu_mse = mean_squared_error(test_cpu, cpu_predictions)
cpu_mae = mean_absolute_error(test_cpu, cpu_predictions)
print(f"CPU Mean Squared Error: {cpu_mse}")
print(f"CPU Mean Absolute Error: {cpu_mae}")

# Evaluate Memory usage model
memory_predictions = memory_model_fit.forecast(steps=len(test_memory))
memory_mse = mean_squared_error(test_memory, memory_predictions)
memory_mae = mean_absolute_error(test_memory, memory_predictions)
print(f"Memory Mean Squared Error: {memory_mse}")
print(f"Memory Mean Absolute Error: {memory_mae}")

# Plot CPU usage predictions
plt.figure(figsize=(10,6))
plt.plot(test_cpu.index, test_cpu, label='Actual CPU Usage')
plt.plot(test_cpu.index, cpu_predictions, label='Predicted CPU Usage', linestyle='--')
plt.legend()
plt.show()

# Plot Memory usage predictions
plt.figure(figsize=(10,6))
plt.plot(test_memory.index, test_memory, label='Actual Memory Usage')
plt.plot(test_memory.index, memory_predictions, label='Predicted Memory Usage', linestyle='--')
plt.legend()
plt.show()
